In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

C:\Users\tiago\AppData\Local\Temp\ipykernel_25972\3777615979.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


# Lab | Natural Language Processing
### SMS: SPAM or HAM

### Let's prepare the environment

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer

- Read Data for the Fraudulent Email Kaggle Challenge
- Reduce the training set to speead up development. 

In [3]:
## Read Data for the Fraudulent Email Kaggle Challenge
data = pd.read_csv(r"C:\Users\tiago\OneDrive\Ambiente de Trabalho\NovaPasta\lab-natural-language-processing\data\kg_train.csv",encoding='latin-1')

# Reduce the training set to speed up development. 
# Modify for final system
data = data.head(1000)
print(data.shape)
data.fillna("",inplace=True)

(1000, 2)


### Let's divide the training and test set into two partitions

In [4]:
from sklearn.model_selection import train_test_split
train_data=data
test_data=pd.read_csv(r"C:\Users\tiago\OneDrive\Ambiente de Trabalho\NovaPasta\lab-natural-language-processing\data\kg_test.csv",encoding='latin-1')

# Split the training set into two parts (80% for training, 20% for validation)
X_train, X_val = train_test_split(train_data, test_size=0.2, random_state=42)

# Split the test set into two parts (80% for testing, 20% for evaluation)
#X_test, X_val_test = train_test_split(test_data, test_size=0.2, random_state=42)



## Data Preprocessing

In [22]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
import string
from nltk.corpus import stopwords
print(string.punctuation)
print(stopwords.words("english")[100:110])
from nltk.stem.snowball import SnowballStemmer
snowball = SnowballStemmer('english')


!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~
['needn', "needn't", 'no', 'nor', 'not', 'now', 'o', 'of', 'off', 'on']


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\tiago\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\tiago\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


## Now, we have to clean the html code removing words

- First we remove inline JavaScript/CSS
- Then we remove html comments. This has to be done before removing regular tags since comments can contain '>' characters
- Next we can remove the remaining tags

In [6]:
import re

# Function to clean HTML content
def clean_html(html_code):
    # Step 1: Remove inline JavaScript and CSS (inside <script> and <style> tags)
    html_code = re.sub(r'<script.*?>.*?</script>', '', html_code, flags=re.DOTALL)
    html_code = re.sub(r'<style.*?>.*?</style>', '', html_code, flags=re.DOTALL)
    
    # Step 2: Remove HTML comments
    html_code = re.sub(r'<!--.*?-->', '', html_code, flags=re.DOTALL)
    
    # Step 3: Remove all remaining HTML tags
    html_code = re.sub(r'<.*?>', '', html_code)
    
    return html_code


- Remove all the special characters
    
- Remove numbers
    
- Remove all single characters
 
- Remove single characters from the start

- Substitute multiple spaces with single space

- Remove prefixed 'b'

- Convert to Lowercase

In [7]:
print(X_train.columns)

Index(['text', 'label'], dtype='object')


In [8]:
# Apply the clean_html function to the relevant column of X_train

X_train['cleaned_content'] = X_train['text'].apply(clean_html)
X_val['cleaned_content'] = X_val['text'].apply(clean_html)

In [9]:
def clean_text(text):
    # 1. Remove special characters (excluding alphabets and spaces)
    text = re.sub(r'[^a-zA-Z\s]', '', text)

    # 2. Remove numbers
    text = re.sub(r'\d+', '', text)

    # 3. Remove all single characters
    text = re.sub(r'\b\w\b', '', text)

    # 4. Remove single characters from the start
    text = re.sub(r'^\b\w\b', '', text)

    # 5. Substitute multiple spaces with a single space
    text = re.sub(r'\s+', ' ', text)

    # 6. Remove prefixed 'b' (if it's part of a string like 'bhello')
    text = re.sub(r'^\bb', '', text)

    # 7. Convert to lowercase
    text = text.lower()

    # Remove any leading or trailing spaces
    text = text.strip()

    return text

In [10]:
# Apply the clean_text function to the relevant column (replace 'text_column' with actual column name)

X_train['cleaned_content'] = X_train['cleaned_content'].apply(clean_text)
X_val['cleaned_content'] = X_val['cleaned_content'].apply(clean_text)

## Now let's work on removing stopwords
Remove the stopwords.

In [11]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import string

def remove_punctuation(sentence):
    """Remove punctuation from a tokenized sentence."""
    return [word for word in sentence if word not in string.punctuation]


def remove_stopwords(sentence):
    """Remove stopwords from a list of tokens."""
    return [word for word in sentence if word not in stopwords.words('english')]



In [12]:
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')
X_train['cleaned_content'] = X_train['cleaned_content'].apply(
    lambda x: word_tokenize(x) if isinstance(x, str) else x)

X_train['cleaned_content'] = X_train['cleaned_content'].apply(remove_punctuation)
X_train['cleaned_content'] = X_train['cleaned_content'].apply(remove_stopwords)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\tiago\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [13]:
X_val['cleaned_content'] = X_val['cleaned_content'].apply(
    lambda x: word_tokenize(x) if isinstance(x, str) else x)

X_val['cleaned_content'] = X_val['cleaned_content'].apply(remove_punctuation)
X_val['cleaned_content'] = X_val['cleaned_content'].apply(remove_stopwords)

## Tame Your Text with Lemmatization
Break sentences into words, then use lemmatization to reduce them to their base form (e.g., "running" becomes "run"). See how this creates cleaner data for analysis!

In [14]:
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

def lemmatize_tokens(tokens):
    """Apply lemmatization to a list of tokens."""
    return [lemmatizer.lemmatize(word) for word in tokens]

lemmatized_X_train=X_train['cleaned_content'].apply(lemmatize_tokens)
lemmatized_X_val=X_val['cleaned_content'].apply(lemmatize_tokens)

# Print the original filtered_X_train and their lemmatized versions
print("Original Tokens:", X_train)



Original Tokens:                                                   text  label  \
29   ----------- REGARDS, MR NELSON SMITH.KINDLY RE...      1   
535  I have not been able to reach oscar this am. W...      0   
695  ; Huma Abedin B6I'm checking with Pat on the 5...      0   
557  I can have it announced here on Monday - can't...      0   
836      BANK OF AFRICAAGENCE SAN PEDRO14 BP 1210 S...      1   
..                                                 ...    ...   
106  7653 2612ADAMA IBRAHIM________________________...      1   
270             What does that mean for our schedules?      0   
860  Dear Friend,My Compliment to you,I guess this ...      1   
435  Dear PRESIDENT=2FDIRECTOR=2C My name is Mr=2E ...      1   
102  Let me know if today or tomorrow works for you...      0   

                                       cleaned_content  
29   [regards, mr, nelson, smithkindly, reply, priv...  
535  [able, reach, oscar, supposed, send, pdb, rece...  
695  [huma, abedin, bim, checki

In [15]:
print("lemmatized Tokens train:", lemmatized_X_train)
print("lemmatized Tokens Val:", lemmatized_X_val)

lemmatized Tokens train: 29     [regard, mr, nelson, smithkindly, reply, priva...
535    [able, reach, oscar, supposed, send, pdb, rece...
695    [huma, abedin, bim, checking, pat, work, jack,...
557                     [announced, monday, cant, today]
836    [bank, africaagence, san, pedro, bp, san, pedr...
                             ...                        
106    [adama, ibrahimtout, savoir, sur, la, scurit, ...
270                                     [mean, schedule]
860    [dear, friendmy, compliment, youi, guess, lett...
435    [dear, presidentfdirectorc, name, mre, micheal...
102    [let, know, today, tomorrow, work, would, rath...
Name: cleaned_content, Length: 800, dtype: object
lemmatized Tokens Val: 521    [dear, sirc, wish, go, offer, consider, partne...
737    [take, mind, balkan, second, see, great, plug,...
740                          [pls, keep, update, coming]
660    [christ, bethel, hospital, rue, aboboteabidjan...
411    [sbwhoeopfriday, february, amhre, bravo,

## Bag Of Words
Let's get the 10 top words in ham and spam messages (**EXPLORATORY DATA ANALYSIS**)

In [16]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
# Convert each list of tokens into a string
lemmatized_X_train_str = [" ".join(words) for words in lemmatized_X_train if len(words) > 0]  
lemmatized_X_val_str = [" ".join(words) for words in lemmatized_X_val if len(words) > 0]  
#  Fix: Remove empty lists
# Step 1: Initialize the CountVectorizer
vectorizer = CountVectorizer()

# Step 2: Fit and transform the corpus into a BoW representation
bow_matrix = vectorizer.fit_transform(lemmatized_X_train_str)

# Get the vocabulary (words)
words = vectorizer.get_feature_names_out()

#Convert Sparse Matrix to Array for Better Visualization
bow_array = pd.DataFrame(bow_matrix.toarray(), columns=vectorizer.get_feature_names_out())

# Print Results
print("Vocabulary (Words in BoW):\n", words)
print("BoW Matrix (Word Frequency Representation):\n", bow_array)

Vocabulary (Words in BoW):
 ['aac' 'aaclocated' 'aae' ... 'zumadirector' 'zumae' 'zurich']
BoW Matrix (Word Frequency Representation):
      aac  aaclocated  aae  aag  aaronovitchon  abacha  abachabefore  abachac  \
0      0           0    0    0              0       0             0        0   
1      0           0    0    0              0       0             0        0   
2      0           0    0    0              0       0             0        0   
3      0           0    0    0              0       0             0        0   
4      0           0    0    0              0       0             0        0   
..   ...         ...  ...  ...            ...     ...           ...      ...   
779    0           0    0    0              0       0             0        0   
780    0           0    0    0              0       0             0        0   
781    0           0    0    0              0       0             0        0   
782    0           0    0    0              0       0           

## Extra features

In [17]:
#convert this varable like we did before 
#X_val_str = [" ".join(words) for words in X_val]
#X_val_df= pd.DataFrame({'preprocessed_text': X_val_str})

# Convert lists into DataFrames (ENSURE THIS STEP)
#bow_array_X_train = pd.DataFrame(bow_array, columns=vectorizer.get_feature_names_out())  # Convert BoW to DataFrame

#  Add preprocessed text to DataFrame (so `.str.contains()` works)
#bow_array_X_train['preprocessed_text'] = lemmatized_X_train_str  # Ensure original text is available


In [18]:
# We add to the original dataframe two additional indicators (money symbols and suspicious words).
money_simbol_list = "|".join(["euro","dollar","pound","€","$"])
suspicious_words = "|".join(["free","cheap","sex","money","account","bank","fund","transfer","transaction","win","deposit","password"])

X_train['money_mark'] = X_train['cleaned_content'].str.contains(money_simbol_list, regex=True, na=False) * 1
X_train['suspicious_words'] = X_train['cleaned_content'].str.contains(suspicious_words, regex=True, na=False) * 1
X_train['text_len'] = X_train['cleaned_content'].apply(lambda x: len(x))

X_val['money_mark'] = X_val['cleaned_content'].str.contains(money_simbol_list, regex=True, na=False) * 1
X_val['suspicious_words'] = X_val['cleaned_content'].str.contains(suspicious_words, regex=True, na=False) * 1
X_val['text_len'] = X_val['cleaned_content'].apply(lambda x: len(x))

X_train.head()


,text,label,cleaned_content,money_mark,suspicious_words,text_len
29,"----------- REGARDS, MR NELSON SMITH.KINDLY RE...",1,"[regards, mr, nelson, smithkindly, reply, priv...",0,0,8
535,I have not been able to reach oscar this am. W...,0,"[able, reach, oscar, supposed, send, pdb, rece...",0,0,7
695,; Huma Abedin B6I'm checking with Pat on the 5...,0,"[huma, abedin, bim, checking, pat, work, jack,...",0,0,14
557,I can have it announced here on Monday - can't...,0,"[announced, monday, cant, today]",0,0,4
836,BANK OF AFRICAAGENCE SAN PEDRO14 BP 1210 S...,1,"[bank, africaagence, san, pedro, bp, san, pedr...",0,0,140


## How would work the Bag of Words with Count Vectorizer concept?

In [19]:
# Steps to Implement BoW with CountVectorizer
#Prepare the text data (list of sentences/documents).
#Initialize CountVectorizer from sklearn.feature_extraction.text.
#Fit and transform the text data into a BoW matrix.
#Convert the sparse matrix to an array for better readability.
#Display the words and the BoW matrix.


## TD-IDF

- Load the vectorizer

- Vectorize all dataset

- print the shape of the vetorized dataset

In [24]:
# Suppose lemmatized_X_train is a Series of token lists
valid_mask = lemmatized_X_train.apply(lambda tokens: len(tokens) > 0)
filtered_X_train = X_train[valid_mask]
filtered_lemmatized_train = lemmatized_X_train[valid_mask]

In [27]:
# Create a mask based on whether the cleaned content yields tokens
valid_mask = lemmatized_X_val.apply(lambda tokens: len(tokens) > 0)
filtered_X_val = X_val[valid_mask]
filtered_lemmatized_val = lemmatized_X_val[valid_mask]

# Convert token lists to strings without dropping any rows (they are already filtered)
lemmatized_X_val_str = [" ".join(words) for words in filtered_lemmatized_val]

In [25]:
#  Initialize TF-IDF Vectorizer
tfidf_vectorizer = TfidfVectorizer(max_features=5000)

# Convert BoW into TF-IDF
X_train_tfidf = tfidf_vectorizer.fit_transform([" ".join(words) for words in filtered_lemmatized_train])
X_val_tfidf = tfidf_vectorizer.transform(lemmatized_X_val_str)

# Get the vocabulary (words)
words_tfidf = tfidf_vectorizer.get_feature_names_out()


# Convert Sparse Matrix to DataFrame for Better Visualization

X_train_tfidf_df = pd.DataFrame(X_train_tfidf.toarray(), columns=tfidf_vectorizer.get_feature_names_out())
X_val_tfidf_df = pd.DataFrame(X_val_tfidf.toarray(), columns=tfidf_vectorizer.get_feature_names_out())

# Print Results
print("Vocabulary (Words in TF-IDF):\n", words_tfidf)
print("TF-IDF Matrix (Word Weight Representation):\n", X_train_tfidf_df)


Vocabulary (Words in TF-IDF):
 ['aaclocated' 'aae' 'abacha' ... 'zone' 'zuhair' 'zuma']
TF-IDF Matrix (Word Weight Representation):
      aaclocated  aae  abacha  abachac  abachae  abad  abandoned  abbas  abdul  \
0           0.0  0.0     0.0      0.0      0.0   0.0        0.0    0.0    0.0   
1           0.0  0.0     0.0      0.0      0.0   0.0        0.0    0.0    0.0   
2           0.0  0.0     0.0      0.0      0.0   0.0        0.0    0.0    0.0   
3           0.0  0.0     0.0      0.0      0.0   0.0        0.0    0.0    0.0   
4           0.0  0.0     0.0      0.0      0.0   0.0        0.0    0.0    0.0   
..          ...  ...     ...      ...      ...   ...        ...    ...    ...   
779         0.0  0.0     0.0      0.0      0.0   0.0        0.0    0.0    0.0   
780         0.0  0.0     0.0      0.0      0.0   0.0        0.0    0.0    0.0   
781         0.0  0.0     0.0      0.0      0.0   0.0        0.0    0.0    0.0   
782         0.0  0.0     0.0      0.0      0.0   0.0     

## And the Train a Classifier?

In [28]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

# Assuming your training and validation DataFrames include a 'label' column.
# If the column name is different, adjust accordingly.

# Extract labels from your training and validation datasets
y_train = filtered_X_train['label']
y_val_filtered = filtered_X_val['label']

# Initialize the Logistic Regression classifier
logreg = LogisticRegression(max_iter=1000)

# Train the classifier using the TF-IDF features
logreg.fit(X_train_tfidf, y_train)

val_predictions = logreg.predict(X_val_tfidf)

print(classification_report(y_val_filtered, val_predictions))

# Print out the first 3 predictions from the validation set
print("First 5 predictions:", val_predictions[:5])




              precision    recall  f1-score   support

           0       0.95      1.00      0.97       124
           1       1.00      0.91      0.95        75

    accuracy                           0.96       199
   macro avg       0.97      0.95      0.96       199
weighted avg       0.97      0.96      0.96       199

First 5 predictions: [1 0 0 1 0]


### Extra Task - Implement a SPAM/HAM classifier

https://www.kaggle.com/t/b384e34013d54d238490103bc3c360ce

The classifier can not be changed!!! It must be the MultinimialNB with default parameters!

Your task is to find the **best feature representation**.

You can work with teams of two persons (recommended).

In [ ]:
# Your code